## Importing Libraries and Display Settings ##

We will need some standard Python libraries for data processing - Pandas and Numpy. 

Also, we will need particular functions from Bokeh data vizualization library. In order to maintain appropriate Jupyter Notebook style, all the uploads for subsequent chapters will be effected in the current section. Where necessary, some additional clarifications will be added.

In [16]:
#Usual imports for data processing
import pandas as pd
import numpy as np

#Bokeh libraries and modules
from bokeh.io import  show, reset_output, output_notebook, export_png
from bokeh.plotting import figure
from bokeh.models import Range1d
from bokeh.layouts import gridplot

#Setting visualizations' display to the in-notebook mode
output_notebook()

#Setting Bokeh's visualtization toolset - a set of functional tools attached to every visual
#Their names are pretty self-explanatory
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

Loading BokehJS ...

## Loading the Dataset ##

In [17]:
data = pd.read_csv('../data/data_clean.csv')

## Simple Bar Charts ##

It is prettry straight-forward to draw bar charts with Bokeh. As usual, we need to specify a type of gart (or chose a glyph) and pass the data to the plotting function.

### Vertical Bar Charts ###

Let's create a vertical bar chart showing changes in measles occurencies in the US over the years 2000-2015.

In [18]:
# Creating a list of categories
years = data[data['country']=='United States of America']['year']

#Creating the list of values
values = data[data['country']=='United States of America']['measles']

# Initializing the plot
p = figure( plot_height=300, 
           title="Measles in the USA 2000-2015",
          tools=TOOLS)

#Plotting
p.vbar(years,                            #categories
      top = values,                      #bar heights
       width = .9,
       fill_alpha = .5,
       fill_color = 'salmon',
       line_alpha = .5,
       line_color='green',
       line_dash='dashed'
      
  )

p.xaxis.axis_label="Years"
p.yaxis.axis_label="Measles stats"

show(p)

### Horizontal Bar Charts ###

Absolutely in the same fashion we could create horizontal bar charts. Let's use reported polio rates for Tanzania in 2000-2015 for illustration purposes.

In [22]:
# Creating a list of categories
years = data[data['country']=='Argentina']['year']

#Creating the list of values
values = data[data['country']=='Argentina']['polio'].values

# Initializing the plot
p = figure( plot_height=300, 
           title="Polio in the Argentina 2000-2015")

#Plotting
p.hbar(years,
       left = 0,
       right = values,
       height = .9,
       fill_color = 'azure',
       line_color='green',
       line_alpha=.5
      
  )

p.xaxis.axis_label="Years"
p.yaxis.axis_label="Polio stats"


show(p)

All these operation look very logical and simple, but their functionality is also limited.

## Styling Bar Charts ##

In order to add much more style, interactivity and attractiveness to our bar charts let's study how to use different pallettes, add labels and hover over our bar charts. In order to illustrate corrsponding techniques let's pick a country at each continent.

In [23]:
countries = ['France', 'Canada', 'Brazil', 'Turkey', 'Australia']

Let's look at the corrsponding statistics for these countries.

### Using Pallettes ###

For a list of some available pallettes please visit [Bokeh pallettes documentation]( https://docs.bokeh.org/en/latest/docs/reference/palettes.html). In order to use any of them with Bokeh we need to import them specifically.

In [38]:
from bokeh.palettes import Spectral5

#Creating the list of values
categories = 



p = figure( plot_height=400, 
           title="Measles in the world in 2015",
          tools=TOOLS)

#Plotting
p.vbar(categories,                            #categories
      top = values,                      #bar heights
       width = .9,
       fill_alpha = .5,
       fill_color = 'salmon',
       line_alpha = .5,
       line_color='green',
       line_dash='dashed'
      
  )

p.xaxis.axis_label="Countries"
p.yaxis.axis_label="Measles stats"

show(p)

IndexError: Item wrong length 1 instead of 2888.

In [35]:
type(categories)

pandas.core.series.Series

In [8]:
data[data['year']==2015][data['country'].isin(countries)]['measles']

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


112      74
352     214
496     195
928     157
2632    342
Name: measles, dtype: int64

In [36]:
data[data['year']==2015][data['country'].isin(countries)][['country','measles']]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,country,measles
112,Australia,74
352,Brazil,214
496,Canada,195
928,France,157
2632,Turkey,342


In [10]:
data[data['country'].isin(countries)]['country'][data['year']==2015]

112     Australia
352        Brazil
496        Canada
928        France
2632       Turkey
Name: country, dtype: object

In [13]:
p=figure()
p.vbar(['a','b','c'],top=[2,3,4])
show(p)